In [10]:
from itertools import combinations
import numpy as np

def bases_iterator(M):
    for Xt in combinations(M.groundset(), M.full_rank()):
        X = frozenset(Xt)
        if M._is_independent(X):
            yield X

In [11]:
r, n = 2, 5
M = matroids.Uniform(r, n)
S = (frozenset({0}), frozenset({1}), frozenset({0, 2}),
    frozenset({0, 2}), frozenset({3}))
k = len(S)
P = PolynomialRing(QQ, 'x', k)

In [ ]:
def get_A_matrix(S, bases):
    """
    Calculates the coefficient matrix A(S) for a rank 2 matroid.

    The matrix A(S) is an n x n symmetric matrix where n = len(S).
    - A[i, i] = c_ii(S) = number of bases B such that B is a subset of S_i.
    - A[i, j] = c_ij(S) = number of bases B = {b1, b2} such that
                        (b1 in S_i and b2 in S_j) or (b1 in S_j and b2 in S_i).
                        This corresponds to the coefficient of x_i * x_j in the
                        (denormalized) based cover class polynomial.

    Args:
    S: A list or tuple of sets (or other iterable collections), representing
        the sequence (S_1, ..., S_n). Elements within the sets should be
        hashable.
    bases: An iterable collection of sets or frozensets, where each inner
            set represents a base B of the rank 2 matroid M. All bases B
            must have |B| = 2.

    Returns:
    A NumPy array representing the n x n symmetric matrix A(S).
    Returns None if bases are not all of size 2.
    """
    n = len(S)
    if n == 0:
        return np.zeros((0, 0), dtype=int)

    # Convert S_i to sets for efficient lookup
    S_sets = [set(s_i) for s_i in S]

    # Initialize A matrix
    A = np.zeros((n, n), dtype=int)

    # Check if bases are valid (all size 2) and convert to frozensets
    processed_bases = []
    for B in bases:
        if len(B) != 2:
            print(f"Error: Base {B} does not have size 2. Matroid must be rank 2.")
            return None
        processed_bases.append(frozenset(B))

    # Iterate through each base B in the matroid M
    for B in processed_bases:
        b1, b2 = tuple(B) # Get the two elements of the base

        # Calculate diagonal entries A[i, i] = c_ii(S)
        for i in range(n):
            # Check if B is a subset of S_i
            if B.issubset(S_sets[i]):
                A[i, i] += 1

        # Calculate off-diagonal entries A[i, j] = c_ij(S) for i < j
        for i in range(n):
            for j in range(i + 1, n):
                # Check if one element is in S_i and the other is in S_j
                if (b1 in S_sets[i] and b2 in S_sets[j]) or \
                (b1 in S_sets[j] and b2 in S_sets[i]):
                    A[i, j] += 1
                    A[j, i] += 1 # Maintain symmetry

    return A

bases = list(bases_iterator(M))

A = get_A_matrix(S, bases)

print("Bases:", bases)
print("-" * 20)
print("Input S:", S)
print("-" * 20)
print("Calculated A(S) matrix:")
print(A)

pt

Bases: [frozenset({0, 1}), frozenset({0, 2}), frozenset({0, 3}), frozenset({0, 4}), frozenset({1, 2}), frozenset({1, 3}), frozenset({1, 4}), frozenset({2, 3}), frozenset({2, 4}), frozenset({3, 4})]
--------------------
Input S: (frozenset({0}), frozenset({1}), frozenset({0, 2}), frozenset({0, 2}), frozenset({3}))
--------------------
Calculated A(S) matrix:
[[0 1 1 1 1]
 [1 0 2 2 1]
 [1 2 1 1 2]
 [1 2 1 1 2]
 [1 1 2 2 0]]
